# Map your IBM Cloud data

- last modified: July, 2016
- author: [Raj Singh](https://developer.ibm.com/clouddataservices/author/rrsingh/)
- original: https://github.com/ibm-cds-labs/open-data/blob/master/samples/cartodb.ipynb
- blog post: [Map your IBM Cloud data](https://developer.ibm.com/clouddataservices/)

## Overview

In [1]:
import pandas as pd, numpy as np, os

<a id="censusdata"></a>
### Get population from Census

Census data is from the 2013 US Census American Community Survey (ACS), 5-year estimates. 

Get the file used here on the [IBM Analytics Exchange](https://console.ng.bluemix.net/data/exchange/). You can get your free, individual "API key" on the data set's detail page:
 
- [Population and Age](https://console.ng.bluemix.net/data/exchange/public/e/united-states-demographic-measures-age)

After getting your API key (your personal URL for the data file), in this script replace, for example, `os.environ['AE_KEY_AGE']` with your URL for the Population and Age data CSV file.

In [2]:
pop_df = pd.read_csv( os.environ['AE_KEY_AGE'], usecols=['GEOID','B01001e1'], dtype={"GEOID": np.str} )
pop_df.columns = ['GEOID','POPULATION']
pop_df = pop_df.set_index('GEOID')
pop_df.sample(10)

,POPULATION
GEOID,
86000US46770,2883
86000US73487,148
86000US89523,31732
86000US78225,14076
86000US02668,2755
86000US21829,880
86000US87724,0
86000US68341,1080
86000US67218,22482


In [3]:
pop_df.describe()

,POPULATION
count,32989.000000
mean,9443.177453
std,13858.010530
min,0.000000
25%,719.000000
50%,2781.000000
75%,12830.000000
max,114734.000000


### Zip codes with more than 100,000 persons

In [4]:
highpop_df = pop_df [(pop_df.POPULATION > 100000)]
highpop_df.describe()

,POPULATION
count,8.000000
mean,106113.375000
std,5637.521642
min,100535.000000
25%,101669.500000
50%,104514.500000
75%,109429.750000
max,114734.000000


### Put it into CartoDB

In [5]:
#uncomment this line on first run to install the cartodb Python module
!pip install --user cartodb
# Set up cartodb module for use
from cartodb import CartoDBAPIKey, CartoDBException
API_KEY = 'yourapikeyhere'
CARTODB_ACCOUNT = 'yourcartodbusernamehere'
cl = CartoDBAPIKey(API_KEY, CARTODB_ACCOUNT)

In [6]:
# output high population zips to a list
ziplist = highpop_df.index.tolist()

In [7]:
updatesql = 'UPDATE zips SET showme = FALSE;'
updatesql += ';'.join(map(lambda x: "UPDATE zips SET showme = TRUE WHERE ge000id LIKE '%s'" %x, ziplist))

In [8]:
updatesql

"UPDATE zips SET showme = FALSE;UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US11220';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US11368';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US60629';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US79936';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US90011';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US90201';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US90650';UPDATE zips SET showme = TRUE WHERE ge000id LIKE '86000US91331'"

In [9]:
try:
    print(cl.sql(updatesql))
except CartoDBException as e:
    print("some error ocurred", e)

{u'fields': {}, u'rows': [], u'total_rows': 1, u'time': 9.149}


In [10]:
%%javascript
element.append('<link rel="stylesheet" href="https://libs.cartocdn.com/cartodb.js/v3/3.15/themes/css/cartodb.css" />');
element.append('<h2>Selected zip codes</h2>');
element.append('<div id="map" style="height:450px;width:768px;padding:0;margin:0"></div>');

require.config({
  paths: {
      cartodblib: 'https://libs.cartocdn.com/cartodb.js/v3/3.15/cartodb'
  }
});

var main = function() {
  cartodb.createVis('map', 'https://rajrsingh.cartodb.com/api/v2/viz/3e4b46a4-3ed3-11e6-bbbe-0e3a376473ab/viz.json', {
      shareable: true,title: true,description: true,search: true,tiles_loader: true,
      center_lat: 40, center_lon: -100, zoom: 3
  })
  .done(function(vis, layers) {
    // layer 0 is the base layer, layer 1 is cartodb layer
    layers[1].setInteraction(true);
    layers[1].on('featureOver', function(e, latlng, pos, data) {
      cartodb.log.log(e, latlng, pos, data);
    });
    var map = vis.getNativeMap(); // get the native map to work with it
  })
  .error(function(err) {
    console.log(err);
  });
}

require(['cartodblib'], main);

<IPython.core.display.Javascript object>